In [1]:
import rasterio
import numpy as np
from rasterio import windows
from itertools import product

# Window loading

In [2]:
def get_windows(src, overlap, width, height, boundless=True):
    offsets = product(range(0, src.meta['width'], width), range(0, src.meta['height'], height))
    full_window = windows.Window(col_off=0, row_off=0, width=src.meta['width'], height=src.meta['height'])
    for col_off, row_off in offsets:
        window = windows.Window(
            col_off=col_off - overlap,
            row_off=row_off - overlap,
            width=width,
            height=height)
        
        if boundless:
            yield window
        else:
            yield window.intersection(full_window)


In [3]:
width = 20
height = 20
overlap = 5

source_data = "../../data/TestDataSet/S2_comp_first.tif"
outdir = "./retiled_images/"

In [4]:
%%time
import xarray as xr
with rasterio.open(source_data) as src:
    window_list = [w for w in get_windows(src, overlap, width, height)]
    arrays = [src.read([1, 2, 3], window=window, boundless=False) for window in window_list]

CPU times: user 19.3 s, sys: 750 ms, total: 20 s
Wall time: 22.6 s


In [8]:
digits = len(str(len(arrays)))
folders = [outdir+'/'.join(a+b for a,b in zip(str(n).zfill(digits)[:-2:2], str(n).zfill(digits)[1:-2:2])) for n in range(len(arrays))]
from pathlib import Path
[Path(f).mkdir(parents=True, exist_ok=True) for f in folders]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [12]:
outpath_list = ['{}/tile_{}.tif'.format(f, c) for f,c in zip(folders, range(len(arrays)))]
outpath_list

['./retiled_images/00/00/tile_0.tif',
 './retiled_images/00/00/tile_1.tif',
 './retiled_images/00/00/tile_2.tif',
 './retiled_images/00/00/tile_3.tif',
 './retiled_images/00/00/tile_4.tif',
 './retiled_images/00/00/tile_5.tif',
 './retiled_images/00/00/tile_6.tif',
 './retiled_images/00/00/tile_7.tif',
 './retiled_images/00/00/tile_8.tif',
 './retiled_images/00/00/tile_9.tif',
 './retiled_images/00/00/tile_10.tif',
 './retiled_images/00/00/tile_11.tif',
 './retiled_images/00/00/tile_12.tif',
 './retiled_images/00/00/tile_13.tif',
 './retiled_images/00/00/tile_14.tif',
 './retiled_images/00/00/tile_15.tif',
 './retiled_images/00/00/tile_16.tif',
 './retiled_images/00/00/tile_17.tif',
 './retiled_images/00/00/tile_18.tif',
 './retiled_images/00/00/tile_19.tif',
 './retiled_images/00/00/tile_20.tif',
 './retiled_images/00/00/tile_21.tif',
 './retiled_images/00/00/tile_22.tif',
 './retiled_images/00/00/tile_23.tif',
 './retiled_images/00/00/tile_24.tif',
 './retiled_images/00/00/tile_25.ti

In [13]:
%%time
import concurrent.futures

with rasterio.open(source_data) as src:
    def write_array(array, window, outpath):
        profile = src.profile
        profile.update(
            transform=src.window_transform(window),
            height=window.height,
            width=window.width)
        with rasterio.open(outpath, 'w', **profile) as dst:
            dst.write(array)
    
    with concurrent.futures.ThreadPoolExecutor(
            max_workers=8
        ) as executor:
            futures = executor.map(write_array, arrays, window_list, outpath_list)

CPU times: user 24min 31s, sys: 3min 11s, total: 27min 42s
Wall time: 1h 14min 28s
